# Bibliothèques

In [342]:
import requests
import pandas as pd
from IPython.display import display, HTML
import pymysql
import mysql.connector


# BASE LILLE

In [ ]:
import requests
import pandas as pd

# URL de l'API
url = "https://data.lillemetropole.fr/data/ogcapi/collections/ilevia:vlille_temps_reel/items?f=json&limit=-1"

# Faire une requête GET à l'API
response = requests.get(url)

# Vérifier si la requête a réussi
if response.status_code == 200:
    data = response.json()
    
    # Vérifier la structure de la réponse JSON
    if 'records' in data:
        records = data['records']
        
        station_v_lille = []
        # Boucle pour extraire les records
        for record in records:
            station = {
                'id': record['@id'],
                'nom': record['nom'],
                'adresse': record['adresse'],
                'commune': record['commune'],
                'etat': record['etat'],
                'type': record['type'],
                'nb_places_dispo': record['nb_places_dispo'],
                'nb_velos_dispo': record['nb_velos_dispo'],
                'etat_connexion': record['etat_connexion'],
                'x': record['x'],
                'y': record['y'],
                'date_modification': record['date_modification']
            }
            station_v_lille.append(station)

        # Convertir la liste de dictionnaires en DataFrame pandas
        df_stations_lille = pd.DataFrame(station_v_lille)
        
        # Afficher le DataFrame sous forme de tableau
        display(df_stations_lille)

# BASE PARIS 

In [ ]:
# URL de l'API
url = "https://opendata.paris.fr/api/explore/v2.1/catalog/datasets/velib-disponibilite-en-temps-reel/records"

# Fonction pour collecter toutes les données de l'API avec pagination
def collect_velib_data(url):
    velib_data = []
    start = 0
    rows_per_page = 100  # Ajuster le nombre de lignes par page si nécessaire

    while True:
        # Faire une requête API avec les paramètres de pagination
        response = requests.get(url, params={"start": start, "rows": rows_per_page})
        
        # Vérifier si la requête a réussi
        if response.status_code == 200:
            try:
                data = response.json()
                
                # Vérifier si la clé "results" est dans la réponse
                if "results" in data:
                    # Boucler à travers chaque enregistrement de station de vélo dans "results"
                    for record in data["results"]:
                        # Extraire les champs pertinents
                        station_info = {
                            "stationcode": record.get("stationcode"),
                            "name": record.get("name"),
                            "is_installed": record.get("is_installed"),
                            "capacity": record.get("capacity"),
                            "numdocksavailable": record.get("numdocksavailable"),
                            "numbikesavailable": record.get("numbikesavailable"),
                            "mechanical_bikes": record.get("mechanical"),
                            "electric_bikes": record.get("ebike"),
                            "is_renting": record.get("is_renting"),
                            "is_returning": record.get("is_returning"),
                            "duedate": record.get("duedate"),
                            "latitude": record["coordonnees_geo"]["lat"],
                            "longitude": record["coordonnees_geo"]["lon"],
                            "nom_arrondissement_communes": record.get("nom_arrondissement_communes"),
                            "code_insee_commune": record.get("code_insee_commune")
                        }
                        # Ajouter l'enregistrement à la liste
                        velib_data.append(station_info)
                    
                    # Si moins d'enregistrements que la limite demandée, nous avons atteint la fin
                    if len(data["results"]) < rows_per_page:
                        break
                    else:
                        start += rows_per_page  # Passer à la page suivante
                else:
                    print("Clé 'results' non trouvée dans la réponse JSON.")
                    break
            except ValueError:
                print("Erreur de décodage JSON.")
                break
        else:
            print(f"La requête a échoué avec le code de statut {response.status_code}.")
            break

    return velib_data

# Collecter les données
velib_data = collect_velib_data(url)

# Convertir la liste de dictionnaires en DataFrame
df_stations_paris = pd.DataFrame(velib_data)

# Afficher le DataFrame sous forme de tableau
display(df_stations_paris)

# BASE TOULOUSE

In [ ]:
# URL de l'API
url = "https://data.toulouse-metropole.fr/api/explore/v2.1/catalog/datasets/stationnement-velo/records?"

def collect_velib_data(url):
    velib_data_toulouse = []
    start = 0
    rows_per_page = 100

    while True:
        try:
            response = requests.get(url, params={"start": start, "rows": rows_per_page})
            data = response.json()

            if "results" in data:
                for record in data["results"]:
                    station_info = {
                        "lib_voie": record.get("lib_voie"),
                        "motdir": record.get("motdir"),
                        "no": record.get("no"),
                        "commune": record.get("commune"),
                        "insee": record.get("insee"),
                        "nb_places": record.get("nb_places"),
                        "coord_x": record.get("coord_x"),
                        "coord_y": record.get("coord_y"),
                        "latitude": record.get("geo_point_2d", {}).get("lat"),
                        "longitude": record.get("geo_point_2d", {}).get("lon")
                    }
                    velib_data_toulouse.append(station_info)

                # Vérifier s'il y a plus d'enregistrements à récupérer
                if len(data["results"]) < rows_per_page:
                    break
                start += rows_per_page
            else:
                break
        except ValueError:
            break
    return velib_data_toulouse

# Collecter les données
velib_data_toulouse = collect_velib_data(url)
print(velib_data_toulouse)

# Convertir la liste de dictionnaires en DataFrame
df_stations_toulouse = pd.DataFrame(velib_data_toulouse)

# Afficher le DataFrame sous forme de tableau
display(df_stations_toulouse)

# Trier et nettoyer les bases

# Lille 

In [ ]:
df_stations_lille.head () 

In [ ]:
# Nettoyage et transformation des données en une seule cellule
df_stations_lille = df_stations_lille.drop(columns=['id', 'etat', 'type', 'etat_connexion', 'date_modification', 'adresse'])
df_stations_lille = df_stations_lille.rename(columns={'x': 'latitude', 'y': 'longitude', 'nom': 'nom_station'})

# Affichage des premières lignes du DataFrame nettoyé
df_stations_lille.head()

# Paris

In [ ]:
df_stations_paris.head ()

In [ ]:
# Nettoyage et transformation des données en une seule cellule
df_stations_paris = df_stations_paris.drop(columns=[
    'numbikesavailable', 'mechanical_bikes', 'electric_bikes', 
    'is_renting', 'is_returning', 'duedate', 'stationcode', 
    'is_installed', 'code_post'
])
df_stations_paris = df_stations_paris.rename(columns={
    'name': 'nom_station',
    'nom_arrondissement_communes': 'commune',
    'code_insee_commune': 'code_post',
    'numdocksavailable': 'nb_velos_dispo',
    'capacity': 'nb_places_dispo'
})

# Modification de la colonne 'nom_station'
df_stations_paris['nom_station'] = df_stations_paris['nom_station'].str.split('-').str[0]

# Affichage des premières lignes du DataFrame nettoyé
df_stations_paris.head()

In [ ]:
df_stations_toulouse.dtypes

# Toulouse 

In [ ]:
df_stations_toulouse.head ()

In [372]:
# Nettoyage et transformation des données en une seule cellule
df_stations_toulouse = df_stations_toulouse.rename(columns={
    'lib_voie': 'nom_station',
    'no': 'nb_velos_dispo',
    'insee': 'code_post',
    'nb_places': 'nb_places_dispo'
})
df_stations_toulouse = df_stations_toulouse.drop(columns=[
    'coord_x', 'coord_y', 'motdir', 'code_post'
])

# Affichage des premières lignes du DataFrame nettoyé
df_stations_toulouse.head()

In [ ]:
missing_data = df_stations_toulouse.isnull().sum()
print(missing_data)

In [374]:
df_stations_toulouse = df_stations_toulouse.dropna()

In [ ]:
df_stations_toulouse.dtypes

In [376]:
# Normaliser la colonne nom_station dans df_stations_toulouse
df_stations_toulouse['nom_station'] = df_stations_toulouse['nom_station'].str.capitalize()

# Normaliser la colonne nom_station dans df_stations_lille
df_stations_lille['nom_station'] = df_stations_lille['nom_station'].str.capitalize()

# Normaliser la colonne nom_station dans df_stations_paris
df_stations_paris['nom_station'] = df_stations_paris['nom_station'].str.capitalize()

In [377]:
# Normaliser la colonne nom_station et commune dans df_stations_toulouse
df_stations_toulouse['nom_station'] = df_stations_toulouse['nom_station'].str.capitalize()
df_stations_toulouse['commune'] = df_stations_toulouse['commune'].str.capitalize()

# Normaliser la colonne nom_station et commune dans df_stations_lille
df_stations_lille['nom_station'] = df_stations_lille['nom_station'].str.capitalize()
df_stations_lille['commune'] = df_stations_lille['commune'].str.capitalize()

# Normaliser la colonne nom_station et commune dans df_stations_paris
df_stations_paris['nom_station'] = df_stations_paris['nom_station'].str.capitalize()
df_stations_paris['commune'] = df_stations_paris['commune'].str.capitalize()

In [ ]:
df_stations_lille.head ()

In [ ]:
df_stations_paris.head ()

In [ ]:
df_stations_toulouse.head ()

In [ ]:
# Concaténer les DataFrames
df_final = pd.concat([df_stations_lille, df_stations_toulouse, df_stations_paris], ignore_index=True)

# Afficher 20 lignes aléatoires du DataFrame final
df_final.sample(n=50)

In [382]:
df_final = df_final[(df_final['latitude'] != 0) & (df_final['longitude'] != 0)]

In [ ]:
# Remplacer les valeurs non numériques par NaN
df_final['nb_velos_dispo'] = pd.to_numeric(df_final['nb_velos_dispo'], errors='coerce')

# Supprimer les lignes avec des NaN dans la colonne nb_velos_dispo
df_final = df_final.dropna(subset=['nb_velos_dispo'])

# Convertir la colonne nb_velos_dispo en int64
df_final['nb_velos_dispo'] = df_final['nb_velos_dispo'].astype('int64')

# Vérifier les types de données pour s'assurer du changement
print(df_final.dtypes)

In [ ]:
print(df_final.dtypes)

# Connecter le df à la base SQL

In [ ]:

import pandas as pd
from sqlalchemy import create_engine

user = 'root'
password = 'root'
host = 'localhost'
unix_socket = '/Applications/MAMP/tmp/mysql/mysql.sock'
database = 'velo'

# Créez l'URL de connexion
connection_string = f"mysql+pymysql://{user}:{password}@{host}/{database}?unix_socket={unix_socket}"

# Créez l'objet engine
engine = create_engine(connection_string)

df_final = df_final

try:
    # Insérez le DataFrame dans la base de données
    df_final.to_sql('velo', con=engine, if_exists='replace', index=False)
    print("DataFrame inséré avec succès dans la base de données")
except Exception as e:
    print(f"Erreur lors de l'insertion du DataFrame : {e}")